# Structure Optimization

### Import modules

In [ ]:
import torch
import os
import sys
sys.path.append(os.path.abspath(".."))

import numpy as np
from ase.build import bulk
from ase.units import GPa
from ase.calculators.emt import EMT
from mattersim.forcefield.potential import MatterSimCalculator
from mattersim.applications.relax import Relaxer

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running MatterSim on {device}")

Running MatterSim on cuda


### Available Models

|                    | mattersim-v1.0.0-1M   | mattersim-v1.0.0-5M     |
| ------------------ | --------------------- | ----------------------- |
| Training Data Size | 3M                    | 6M                      |
| Model Parameters   | 880K                  | 4.5M                    |

In [3]:
# model = 1 or 5 only
model = 5

### Si structure

##### Setup Si structure to relax

In [4]:
# initialize the structure of silicon
si = bulk("Si", "diamond", a=5.43)

print(f"si positions: {si.positions}")

# attach the calculator to the atoms object
si.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)
print(f"\nMatterSimCalculator loaded with {model}M model\n")

si positions: [[0.     0.     0.    ]
 [1.3575 1.3575 1.3575]]


2025-01-31 02:24:08.661 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-01-31 02:24:09.423 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-01-31 02:24:09.681 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model


/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=


MatterSimCalculator loaded with 5M model



In [5]:
print(f"Energy (eV)                 = {si.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {si.get_potential_energy()/len(si)}")
print(f"Forces of first atom (eV/A) = {si.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {si.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {si.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.814007759094238
Energy per atom (eV/atom)   = -5.407003879547119
Forces of first atom (eV/A) = [-5.9837475e-07 -1.0267831e-06 -9.9698082e-07]
Stress[0][0] (x-x) (eV/A^3) = -0.011929347679320034
Stress[0][0] (x-x) (GPa)    = -1.9112921953201294


In [6]:
# initialize the relaxation object
relaxer = Relaxer(
    optimizer="BFGS", # the optimization method
    filter="ExpCellFilter", # filter to apply to the cell
    constrain_symmetry=True, # whether to constrain the symmetry
)

relaxed_structure = relaxer.relax(si, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 02:24:09      -10.814008        0.477481
BFGS:    1 02:24:09      -10.818989        0.015248


BFGS:    2 02:24:09      -10.818996        0.000544


In [7]:
print(f"Energy (eV)                 = {si.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {si.get_potential_energy()/len(si)}")
print(f"Forces of first atom (eV/A) = {si.get_forces()[0]}")
print(f"Stress[0][0] (eV/A^3)       = {si.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (GPa)          = {si.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.81899642944336
Energy per atom (eV/atom)   = -5.40949821472168
Forces of first atom (eV/A) = [-6.6589564e-07 -6.7939982e-07 -9.4901770e-07]
Stress[0][0] (eV/A^3)       = -1.3287519893339393e-05
Stress[0][0] (GPa)          = -0.0021288953721523285


##### Perturb Si structure & perform relaxation

In [8]:
# initialize the structure of silicon
si = bulk("Si", "diamond", a=5.43)

# perturb the structure
si.positions += 0.1 * np.random.randn(len(si), 3)

# attach the calculator to the atoms object
si.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=

In [9]:
print(f"Energy (eV)                 = {si.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {si.get_potential_energy()/len(si)}")
print(f"Forces of first atom (eV/A) = {si.get_forces()[0]}")
print(f"Stress[0][0] (eV/A^3)       = {si.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (GPa)          = {si.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.566696166992188
Energy per atom (eV/atom)   = -5.283348083496094
Forces of first atom (eV/A) = [ 0.31849664 -1.7726748  -1.0068196 ]
Stress[0][0] (eV/A^3)       = -0.023936073152793204
Stress[0][0] (GPa)          = -3.834981679916382


In [10]:
relaxed_structure = relaxer.relax(si, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 02:24:09      -10.566696        2.063371
BFGS:    1 02:24:09      -10.720662        1.141913
BFGS:    2 02:24:09      -10.770748        0.507386
BFGS:    3 02:24:09      -10.787316        0.606034
BFGS:    4 02:24:09      -10.812212        0.424415
BFGS:    5 02:24:09      -10.818182        0.167654
BFGS:    6 02:24:09      -10.818883        0.041141
BFGS:    7 02:24:09      -10.818934        0.020359
BFGS:    8 02:24:09      -10.818970        0.019127
BFGS:    9 02:24:09      -10.818988        0.011502
BFGS:   10 02:24:09      -10.818993        0.004550


In [11]:
print(f"Energy (eV)                 = {si.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {si.get_potential_energy()/len(si)}")
print(f"Forces of first atom (eV/A) = {si.get_forces()[0]}")
print(f"Stress[0][0] (eV/A^3)       = {si.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (GPa)          = {si.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.818992614746094
Energy per atom (eV/atom)   = -5.409496307373047
Forces of first atom (eV/A) = [0.00198698 0.00217223 0.00016647]
Stress[0][0] (eV/A^3)       = 5.1069971464743164e-05
Stress[0][0] (GPa)          = 0.008182311430573463


In [12]:
relaxed_structure[1].cell

Cell([[0.00010035695925305417, 2.7344617642167797, 2.734391787819], [2.734320637096266, 0.00013586045734002681, 2.7343562843209166], [2.734184474565204, 2.7342900981876586, 0.00027202298840665526]])

### C structure

In [13]:
# generate a random 'a' to start with.
a = np.random.uniform(1,10)
a

8.641630779544606

In [14]:
c = bulk('C', 'diamond', a=a)
c.positions

array([[0.        , 0.        , 0.        ],
       [2.16040769, 2.16040769, 2.16040769]])

In [15]:
c.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

In [16]:
print(f"Energy (eV)                 = {c.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {c.get_potential_energy()/len(c)}")
print(f"Forces of first atom (eV/A) = {c.get_forces()[0]}")
print(f"Stress[0][0] (GPa)          = {c.get_stress(voigt=False)[0][0] / GPa}")
print(f"Stress[0][0] (eV/A^3)       = {c.get_stress(voigt=False)[0][0]}")

Energy (eV)                 = -3.382246971130371
Energy per atom (eV/atom)   = -1.6911234855651855
Forces of first atom (eV/A) = [-0. -0. -0.]
Stress[0][0] (GPa)          = -0.07146544009447098
Stress[0][0] (eV/A^3)       = -0.00044605219653490386


In [17]:
relaxed_structure = relaxer.relax(c, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 02:24:09       -3.382247        0.071964
BFGS:    1 02:24:10       -3.382470        0.072855
BFGS:    2 02:24:10       -3.425565        0.378357
BFGS:    3 02:24:10       -3.919960        4.008593
BFGS:    4 02:24:10       -4.852183        0.693593
BFGS:    5 02:24:10       -4.869107        0.045949
BFGS:    6 02:24:10       -4.869249        0.028882
BFGS:    7 02:24:10       -4.869369        0.008988


In [18]:
print(f"Energy (eV)                 = {c.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {c.get_potential_energy()/len(c)}")
print(f"Forces of first atom (eV/A) = {c.get_forces()[0]}")
print(f"Stress[0][0] (GPa)          = {c.get_stress(voigt=False)[0][0] / GPa}")
print(f"Stress[0][0] (eV/A^3)       = {c.get_stress(voigt=False)[0][0]}")

Energy (eV)                 = -4.869368553161621
Energy per atom (eV/atom)   = -2.4346842765808105
Forces of first atom (eV/A) = [-0. -0. -0.]
Stress[0][0] (GPa)          = -0.003771942807361483
Stress[0][0] (eV/A^3)       = -2.3542615454456414e-05


In [19]:
relaxed_structure[1].cell

Cell([[8.796608899443138e-17, 5.757764698709902, 5.757764698709904], [5.757764698709904, 1.3266605662273267e-16, 5.757764698709904], [5.757764698709904, 5.757764698709902, 1.1261345475533858e-16]])